<a href="https://colab.research.google.com/github/trammmd/MaennerML/blob/main/Maenner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install shap
!pip install seaborn

In [66]:
#Connect Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Data Loading


In [67]:
#importing necessery libraries for future analysis of the dataset
import pandas as pd
from tensorflow.keras import models, layers, utils, backend as K
import matplotlib.pyplot as plt
import shap
import matplotlib as plt
import seaborn as sns
import folium


In [68]:
#Read data
uBahn_1=pd.read_json("/content/drive/MyDrive/Colab Notebooks/ML4B/uBahn/Ubahn_Rathenauplatz-Rennweg.23-05-23_13-11-53.json")
uBahn_2=pd.read_json("/content/drive/MyDrive/Colab Notebooks/ML4B/uBahn/Ubahn_Rathenauplatz-Wöhrder Wiese.24-05-23_13-32-31.json")
uBahn_3=pd.read_json("/content/drive/MyDrive/Colab Notebooks/ML4B/uBahn/Ubahn_Rennweg-Schoppershof.23-05-23_13-13-24.json")
uBahn_4=pd.read_json("/content/drive/MyDrive/Colab Notebooks/ML4B/uBahn/Ubahn_Wöhrder Wiese-Hbf-Opernhaus.24-05-23_13-33-39.json")
uBahn_5=pd.read_json("/content/drive/MyDrive/Colab Notebooks/ML4B/uBahn/uBahn-2023-05-22_15-59-14 2.json")
uBahn_6=pd.read_json("/content/drive/MyDrive/Colab Notebooks/ML4B/uBahn/uBahn_Bauernfeindstr._-_Langwasser_Mitte-2023-05-24_06-22-01 2.json")
uBahn_7=pd.read_json("/content/drive/MyDrive/Colab Notebooks/ML4B/uBahn/uBahn_Hbf-Friedrich_Ebert_Platz-2023-05-23_11-21-01 2.json")
Fahrrad_1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML4B/Fahrrad_Zip/Location1.csv")
Fahrrad_2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML4B/Fahrrad_Zip/Location2.csv")
Fahrrad_3 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML4B/Fahrrad_Zip/Location3.csv")
Fahrrad_4 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML4B/Fahrrad_Zip/Location4.csv")
Fahrrad_5 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML4B/Fahrrad_Zip/Location5.csv")



In [69]:
#Create dataframes for Ubahn
uBahn_df = [uBahn_1, uBahn_2, uBahn_3, uBahn_4, uBahn_5, uBahn_6, uBahn_7]
uBahn_df  = pd.concat(uBahn_df )

#Create df for Fahrrad

Fahrrad_df= [Fahrrad_1, Fahrrad_2, Fahrrad_3, Fahrrad_4, Fahrrad_5]
Fahrrad_df = pd.concat(Fahrrad_df)

#Feature Engineering


In [70]:
#Extract Location from df
location_uBahn = uBahn_df[uBahn_df['sensor'] == 'Location']

In [71]:
#Add new column for traffic
for i in range(len(location_uBahn)):
  location_uBahn['Type'] = 'uBahn'

for i in range(len(Fahrrad_df)):
    Fahrrad_df['Type'] = 'Fahrrad'
Fahrrad_df.head()

<ipython-input-71-22027c5a7cab>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,time,seconds_elapsed,altitude,speedAccuracy,bearingAccuracy,latitude,altitudeAboveMeanSeaLevel,bearing,horizontalAccuracy,verticalAccuracy,longitude,speed,Type
0,1683706093318377000,-28.847623,359.387531,-1.000000,-1.000000,49.455863,313.797531,-1.000000,11.416650,14.562453,11.126913,-1.000000,Fahrrad
1,1683706150999348200,28.833348,359.336569,0.723846,25.482019,49.455798,313.746569,231.174278,7.715990,8.053696,11.126582,0.503094,Fahrrad
2,1683706208000208400,85.834208,359.309165,1.018800,16.142837,49.455288,313.719165,232.706895,4.840367,3.224188,11.124600,3.399262,Fahrrad
3,1683706265000976100,142.834976,359.716832,0.974201,10.291368,49.453161,314.126832,225.542776,4.762798,3.337708,11.122296,5.012913,Fahrrad
4,1683706323000254500,200.834254,358.420130,1.017401,11.544745,49.451676,312.830130,234.996558,4.778603,3.315040,11.118777,4.727707,Fahrrad


In [72]:
location_uBahn.isnull().sum()

sensor                         0
time                           0
seconds_elapsed                0
z                            379
y                            379
x                            379
bearingAccuracy                0
speedAccuracy                  0
verticalAccuracy               0
horizontalAccuracy             0
speed                          0
bearing                        0
altitude                       0
longitude                      0
latitude                       0
version                      379
device name                  379
recording time               379
platform                     379
appVersion                   379
device id                    379
sensors                      379
sampleRateMs                 379
qz                           379
qy                           379
qx                           379
qw                           379
roll                         379
pitch                        379
yaw                          379
relativeAl

In [73]:
#Drop collumns
location_uBahn = location_uBahn.drop(['x','y','z','version','device name','recording time','platform','appVersion','device id', 'sensors','sampleRateMs','qz','qy','qx','qw','roll','pitch','yaw','relativeAltitude','pressure'], axis=1)

In [74]:
location_uBahn.isnull().sum()

sensor                        0
time                          0
seconds_elapsed               0
bearingAccuracy               0
speedAccuracy                 0
verticalAccuracy              0
horizontalAccuracy            0
speed                         0
bearing                       0
altitude                      0
longitude                     0
latitude                      0
altitudeAboveMeanSeaLevel    67
Type                          0
dtype: int64

In [80]:
#for i in range(len(location_uBahn)):
#  if location_uBahn['altitudeAboveMeanSeaLevel'].isnull():

'''
for i in range(len(location_uBahn)):
  if location_uBahn.isnull():
    print(location_uBahn(i-1))
    print(location_uBahn(i))
    print(location_uBahn(i+1))
'''





'\nfor i in range(len(location_uBahn)):\n  if location_uBahn.isnull():\n    print(location_uBahn(i-1))\n    print(location_uBahn(i))\n    print(location_uBahn(i+1))\n'

In [77]:
import plotly.express as px

**Correlation Matrix**

Its necessary to remove correlated variables to improve your model. One can find correlations using pandas “.corr()” function and can visualize the correlation matrix using plotly express.


Lighter shades represents positive correlation, when one variable increases, the other variable tends to increase

Darker shades represents negative correlation, when one variable increases, the other variable tends to decrease

In [78]:
px.imshow(location_uBahn.corr(),title="Correlation Plot")

<ipython-input-78-2bb7a848ff27>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



To read: [Fundamental Techniques of Feature Engineering for Machine Learning](https://towardsdatascience.com/feature-engineering-for-machine-learning-3a5e293a5114)



#NEXT STEP
Separating labels and predictors

```
X=df.drop('location',axis=1)
y=df['location'].values
```
Splitting train (75%) and test set (25%)

```
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
```
Selecting categorical dataframe in train set
```
X_train_cat=df.select_dtypes(exclude=['number'])
```
Translate transport type into numerical values. For example: auto as 0, bus as 1, ubahn as 2, fahrrad as 3

Applying label encoding for categorical features in train set
```
importing label encoder
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
X_train_cat=X_train_cat.apply(LabelEncoder().fit_transform)
```
Importing ColumnTransformer and OneHotEncoder
```
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
```

Separating numerical and categorical attributes in the train set
```
num_attribs = list (X_train_num)
cat_attribs = list (X_train_cat)
```
Creating a full pipeline: numerical + categorical
```
full_pipeline = ColumnTransformer([("num",num_pipeline,num_attribs),("cat",```OneHotEncoder(handle_unknown='ignore'),cat_attribs)])
```

Fit and transform the train set using the full pipeline
```
X_train_prepd = full_pipeline.fit_transform(X_train)
```


To do:

Location_bus,

Location_fahhrad,

Location_auto

Imputation: Fill in values of altitudeAboveMeanSeaLevel


